In [1]:
from IPython.display import display, HTML
display(HTML("<style>div.container { width:100% !important; }</style>"))

# 합성곱 신경망의 시각화

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [ ]:
model = keras.models.load_model()

#### 
05. 인공지능 알고리즘을 사용하여 인지 기능을 구현하고 
과정과 결과를 주석으로 처리 후 코드와 결과를 증빙하시오.

In [ ]:
# 05-1. Fashion MNIST 데이터 로드
(train_X, train_y), (test_X, test_y) = fashion_mnist.load_data()

print('train_X shape:', train_X.shape)
print('test_X shape:', test_X.shape)

In [ ]:
# 05-2. 데이터 정규화
train_scaled = train_X.reshape(-1, 28, 28, 1) / 255.0  # float type 변경
test_scaled = test_X.reshape(-1, 28, 28, 1)  / 255.0

In [ ]:
print('train_scaled shape:', train_scaled.shape)
print('test_scaled shape:', test_scaled.shape)

In [ ]:
from sklearn.model_selection import train_test_split

train_scaled, val_scaled, train_target, val_target = train_test_split (
        train_scaled, train_y, test_size=0.2, random_state=42)

In [ ]:
print('train_scaled shape:', train_scaled.shape)
print('val_scaled shape:', val_scaled.shape)

In [ ]:
# 05-3. 모델 구성
model = Sequential()

In [ ]:
model.add(Conv2D(32, kernel_size=3, input_shape=(28, 28, 1), padding="same", use_bias=True, activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), padding="same", use_bias=True, activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(10, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, to_file='cnn-architecture.png', dpi=100)

In [ ]:
# 05-4. 모델 컴파일
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("best-cnn-model.h5")

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True) 

In [ ]:
# 05-5. 모델 학습, 모델 저장
history = model.fit(train_scaled, train_target,epochs=10,
                    validation_data=(val_scaled, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb] )

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()
loss_ax.plot(history.history["loss"], "y", label="train_loss")
loss_ax.plot(history.history["val_loss"], "r", label="val_loss")
loss_ax.set_xlabel("epoch")
loss_ax.set_ylabel("loss")
loss_ax.legend(loc="upper left")

acc_ax = loss_ax.twinx()
acc_ax.plot(history.history["accuracy"], "b", label="train_acc")
acc_ax.plot(history.history["val_accuracy"], "g", label="val_acc")
acc_ax.set_ylabel("accuracy")
acc_ax.legend(loc="lower left")

In [ ]:
# 05-6 모델 평가
score = model.evaluate(val_scaled, val_target, verbose=1)
print('val loss:', score[0])
print('val accuracy:', score[1])

In [ ]:
val loss: 0.2226928323507309
val accuracy: 0.9242500066757202
    
손실 값이 낮고 정확도가 높으므로, 모델이 검증 데이터에 대해서 좋은 성능을 보이고 있습니다. 
손실 값이 낮다는 것은 모델이 예측과 실제 값 간의 차이가 적다는 의미이며, 
정확도가 높다는 것은 모델이 많은 데이터를 올바르게 분류했다는 의미입니다.
검증 데이터에 대한 손실과 정확도는 학습 중에 설정한 손실 함수와 
평가 지표에 따라서도 달라질 수 있습니다. 
이러한 결과를 통해 모델이 훈련 데이터에 과적합되지 않고, 
새로운 데이터에 대해서도 일반화될 수 있음을 확인할 수 있습니다.

In [ ]:
from matplotlib.pyplot as plt
plt.imshow(val_scaled[0].reshape(28, 28), cmap="gray_r") 
plt.show()

In [ ]:
pred = model.predict(val_scaled[0:1])
print(pred)

In [ ]:
score = model.evaluate(test_scaled, test_y, verbose=1)
print('val loss:', score[0])
print('val accuracy:', score[1])